In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import warnings
warnings.filterwarnings("ignore")
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from IPython.display import Image, display
import pprint
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing import Annotated, Literal
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
import ast


def build_vector_db():

    def _set_env():
        load_dotenv()
        if "GOOGLE_API_KEY" not in os.environ:
            os.environ["GOOGLE_API_KEY"] = getpass.getpass(
                "Enter your Google AI API key: "
            )

    _set_env()

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        # other params...
    )

    urls = [
        os.environ["moneycontrol"],
        os.environ["economic_times"],
        os.environ["yahoo_fin"],
    ]

    print(urls)

    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=100, chunk_overlap=20
    )
    doc_splits = text_splitter.split_documents(docs_list)

    # Add to vectorDB
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="rag-chroma",
        embedding=embeddings,
        persist_directory = "./agent"
        
        
    )
    retriever = vectorstore.as_retriever()

    retriever_tool = create_retriever_tool(
        retriever,
        "retriver_stock_market_updates",
        "Search and return information about stocks in news whether they are good news or bad news , , bullish and bearnish newses as well",
    )

    tools = [retriever_tool]

    return retriever_tool, retriever, tools, vectorstore


/home/kgvt/project_agent_final/project_agent/project_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# retriever_tool, retriever, tools, vectorstore = build_vector_db()

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [2]:
vector_store = Chroma(
    collection_name="rag-chroma",
    persist_directory="./agent",  # Where to save data locally, remove if not necessary
)

/tmp/ipykernel_931/3638167432.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [3]:
vector_store

In [7]:
retriever = vector_store.as_retriever()

In [8]:
retriever.invoke("is bitcoin bullish?")

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.

In [12]:
retriever_2 = vector_store.as_retriever(embedding_function=embeddings.embed_query)

In [14]:
retriever_2.invoke("is bitcoin bullish?")

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.

In [16]:
vectorstore_2 = Chroma(collection_name="rag-chroma", persist_directory="./agent")

In [17]:
retriever_2 = vectorstore_2.as_retriever(embedding_function=embeddings.embed_query)

In [21]:
retriever_2.invoke("is bitcoin bullish?")

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.